In [1]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
DATASET_PATH = r'C:\Users\leewa\OneDrive\Desktop\model2\SDFVD'
REAL_PATH = os.path.join(DATASET_PATH, "videos_real")
FAKE_PATH = os.path.join(DATASET_PATH, "videos_fake")

IMG_SIZE = (224, 224)  
FRAME_COUNT = 30       



In [3]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return np.zeros((FRAME_COUNT, IMG_SIZE[0], IMG_SIZE[1], 3))  # Return blank frames
    
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // FRAME_COUNT)

    for i in range(0, total_frames, frame_interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, IMG_SIZE)
            frame = tf.keras.applications.resnet50.preprocess_input(frame)
            frames.append(frame)

    
    blank_frame = tf.keras.applications.resnet50.preprocess_input(
        np.zeros((IMG_SIZE[0], IMG_SIZE[1], 3), dtype=np.float32)
    )
    while len(frames) < FRAME_COUNT:
        frames.append(blank_frame)

    return np.array(frames[:FRAME_COUNT])


In [4]:
def load_dataset():
    x, y = [], []
    for label, path in [(0, REAL_PATH), (1, FAKE_PATH)]:
        videos = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".mp4")]
        for video in videos[:100]:  # Limit dataset size
            frames = extract_frames(video)
            x.append(frames)
            y.append(label)
    return np.array(x), np.array(y)


In [6]:
# Check if directories exist and create them if they don't
if not os.path.exists(REAL_PATH):
	os.makedirs(REAL_PATH)
if not os.path.exists(FAKE_PATH):
	os.makedirs(FAKE_PATH)

# Verify that there are video files in the directories
real_videos = [f for f in os.listdir(REAL_PATH) if f.endswith('.mp4')] if os.path.exists(REAL_PATH) else []
fake_videos = [f for f in os.listdir(FAKE_PATH) if f.endswith('.mp4')] if os.path.exists(FAKE_PATH) else []

print(f"Found {len(real_videos)} real videos and {len(fake_videos)} fake videos")

if len(real_videos) > 0 and len(fake_videos) > 0:
	x, y = load_dataset()
	print(f"Dataset loaded: X shape {x.shape}, y shape {y.shape}")
else:
	print("Error: No video files found in the directories.")
	print(f"Please ensure .mp4 files are present in:\n{REAL_PATH}\n{FAKE_PATH}")


Found 0 real videos and 0 fake videos
Error: No video files found in the directories.
Please ensure .mp4 files are present in:
C:\Users\leewa\OneDrive\Desktop\model2\SDFVD\videos_real
C:\Users\leewa\OneDrive\Desktop\model2\SDFVD\videos_fake


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],  
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_frames = [datagen.random_transform(frame) for frame in x.reshape(-1, 224, 224, 3)]
x = np.array(augmented_frames).reshape(-1, 30, 224, 224, 3)
print("Data augmentation applied.")


Data augmentation applied.


In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False)


for layer in base_model.layers[:-10]:  
    layer.trainable = False
for layer in base_model.layers[-10:]:  
    layer.trainable = True


In [ ]:
model = Sequential([
    TimeDistributed(base_model, input_shape=(30, 224, 224, 3)),
    TimeDistributed(GlobalAveragePooling2D()),
    LSTM(64, return_sequences=True),
    Dropout(0.2),  # Prevent overfitting
    LSTM(64),
    Dropout(0.2),  # Prevent overfitting
    Dense(64, activation="relu"),
    Dropout(0.3),  # Prevent overfitting
    Dense(1, activation="sigmoid")
])

model.summary()


c:\Users\leewa\anaconda3\a cute anaconda\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 30, 7, 7, 2048) │    23,587,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 2048)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 30, 64)         │       540,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,165,889 (92.19 MB)

 Trainable params: 5,043,841 (19.24 MB)

 Non-trainable params: 19,122,048 (72.94 MB)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
model.fit(x, y, epochs=5, batch_size=8, validation_split=0.25)
print("Stage 1 training complete.")


Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 481s 28s/step - accuracy: 0.5697 - loss: 0.6815 - val_accuracy: 0.0000e+00 - val_loss: 1.2270
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 117s 12s/step - accuracy: 0.6000 - loss: 0.7194 - val_accuracy: 0.0000e+00 - val_loss: 1.0708
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 111s 11s/step - accuracy: 0.6250 - loss: 0.6866 - val_accuracy: 0.0000e+00 - val_loss: 1.1274
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 113s 12s/step - accuracy: 0.6806 - loss: 0.5674 - val_accuracy: 0.0000e+00 - val_loss: 1.5199
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 112s 11s/step - accuracy: 0.6961 - loss: 0.6035 - val_accuracy: 0.0000e+00 - val_loss: 1.3037
Stage 1 training complete.


In [ ]:
for layer in base_model.layers[-10:]:  
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss="binary_crossentropy", metrics=["accuracy"])



In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model_resnet_lstm.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model.fit(x, y, epochs=15, batch_size=4, validation_split=0.25, callbacks=[checkpoint])
print("Training complete. Model saved as 'best_model_resnet_lstm2.h5'.")


Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.6372 - loss: 0.6288 
Epoch 1: val_loss improved from inf to 1.32689, saving model to best_model_resnet_lstm.h5


20/20 ━━━━━━━━━━━━━━━━━━━━ 672s 20s/step - accuracy: 0.6382 - loss: 0.6282 - val_accuracy: 0.0000e+00 - val_loss: 1.3269
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.6151 - loss: 0.6060 
Epoch 2: val_loss did not improve from 1.32689
20/20 ━━━━━━━━━━━━━━━━━━━━ 362s 18s/step - accuracy: 0.6177 - loss: 0.6049 - val_accuracy: 0.0000e+00 - val_loss: 1.3926
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.6688 - loss: 0.6135 
Epoch 3: val_loss did not improve from 1.32689
20/20 ━━━━━━━━━━━━━━━━━━━━ 378s 19s/step - accuracy: 0.6689 - loss: 0.6138 - val_accuracy: 0.0000e+00 - val_loss: 1.4120
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7046 - loss: 0.6093 
Epoch 4: val_loss did not improve from 1.32689
20/20 ━━━━━━━━━━━━━━━━━━━━ 358s 18s/step - accuracy: 0.7030 - loss: 0.6102 - val_accuracy: 0.0000e+00 - val_loss: 1.4152
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.7392 - loss: 0.5476 
Epoch 5: val_loss did not improve f

In [ ]:
model.save("deepfake_detector_resnet_lstm.keras")
print("Model saved successfully!")


Model saved successfully!
